astro_101 placeholder for (course name)

# Una Reevaluación del Primer Sistema Planetario Confirmado Descubierto Mediante el Método de Velocidades Radiales, 51 Pegasi b

Este cuaderno le ayudará a instalar todo lo que necesita para realizar la reducción y análisis de los datos de 51 Pegasi.

**Asegúrese de leerlo bien.**



En este proyecto, el estudiante obtendrá y analizará los datos de series temporales de velocidad radial de alta precisión (PRV) más recientes y actualizados sobre el primer sistema planetario confirmado, 51 Pegasi b. En 1995, el campo de los estudios de planetas extrasolares se lanzó hacia un nuevo paradigma con el descubrimiento y posterior confirmación del planeta 51 Pegasi b (Mayor & Queloz 1995, Nature). Este sistema ha seguido siendo observado en los años posteriores mediante instrumentos PRV, con el objetivo de refinar aún más los parámetros planetarios y buscar planetas adicionales. Aquí usaremos nuestro código de análisis bayesiano EMPEROR para examinar este conjunto de datos denso y de largo plazo, para mejorar aún más nuestra comprensión de este sistema y cómo se pueden usar los métodos bayesianos para buscar señales Doppler en datos con ruido.

## ¿Qué se hará?
El estudiante debe realizar un análisis estadístico de los datos observados, incluidas estimaciones de parámetros, modelización de ruido e inferencia estadística, proporcionando respuestas coherentes a algunas preguntas amplias que se enumeran a continuación. En comparación con el formato típico en papel para el informe escrito, el estudiante se centrará principalmente en los resultados científicos del estudio realizado. Se espera que se incluya discusión sobre los valores de los parámetros de EMPEROR, en particular de las características de la órbita del planeta 51 Pegasi b. ¿Qué tipo de orbita tiene?, ¿qué masa mínima tiene el planeta?, ¿qué tipo de planeta es? Existe la posibilidad de tener otros planetas en el sistema también?  Explica qué sabes de la estadística bayesiana y probabilidades.

## Empezando
El código EMPEROR se ha trasladado a Google Collab, de modo que las pruebas de 51 Pegasi se pueden ejecutar directamente en los servidores de Google. Esto hace que ejecutar EMPEROR por primera vez sea bastante sencillo y todos los resultados se pueden descargar desde el sitio de Collab.
Puede encontrar el Proyecto de colaboración EMPEROR en el siguiente enlace, y todos los archivos de ayuda deberían facilitar el proceso para comenzar a ajustar los datos del RV.

Colab EMPEROR: https://colab.research.google.com/drive/1K4SvaK94-b30K0FCKXtFvCS9t0J_v3Xh?usp=sharing

EMPEROR GitHub: https://github.com/ReddTea/astroEMPEROR

# 1 Setup
En esta sección procederemos a dejar listo todo lo necesario para realizar la actividad.

Lo primero que debe realizar es descargar los archivos necesarios para la actividad. Para esto, debe:

1.   enlazar GD al colab
2.   descargar archivos a su GD


El primer paso que debe tomar es conectar su Google Drive (hágalo con el asociado a su correo mail.udp.cl). Esto lo puede hacer de dos modos distintos:

1. Ejecute el código de las dos celdas siguientes (haciendo click sobre el icono play). La segunda celda abrirá un pop-up en su browser, donde le pide confirmar permisos para su drive.

2. Alternativamente, en el lado izquierdo aprete el ícono que tiene forma de carpeta. En el menú que se abre, haga click en el ícono de Google Drive (carpeta con un símbolo parecido al del reciclaje) y de permiso para su conexión.

In [1]:
import os
from google.colab import drive

In [2]:
# conectar drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Cambiamos el directorio de trabajo al de Google Drive.
os.chdir('drive/MyDrive')

In [7]:
#Descargamos las rutinas desde el repositorio de Github.
!git clone https://github.com/ReddTea/udp_astro_101


# La siguiente linea es para actualizar el repositorio.
#!git pull https://github.com/ReddTea/udp_astro_101

Cloning into 'udp_astro_101'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 22 (delta 5), reused 17 (delta 3), pack-reused 0
Receiving objects: 100% (22/22), 15.86 KiB | 1.32 MiB/s, done.
Resolving deltas: 100% (5/5), done.


# 2 Tests
Nos aseguraremos que las rutinas pueden correr sin problemas con el siguiente test. Para esto es necesario:


1.   Ubicarse en la carpeta de trabajo
2.   Instalar las librerias necesarias
3.   Correr los test



In [ ]:
#Cambiamos el directorio de trabajo a la carpeta que acabamos de descargar
os.chdir('udp_astro_101')

### 2.1 Instalar librerias de python

In [9]:
# Instalaremos las librerías de python necesarias
%%capture
import numpy as np
try:
    import reddemcee
except ModuleNotFoundError:
    !pip3 install git+https://github.com/ReddTea/reddemcee.git
    import reddemcee

try:
    import astroemperor as emp
except ModuleNotFoundError:
    !pip3 install git+https://github.com/ReddTea/astroemperor.git
    import astroemperor as emp

### 2.2 Reddemcee
En este pequeño bloque haremos un test rápido con reddemcee, el sampler que utiliza emperor como engine.

Para ello definimos un likelihood y un prior, además de los metaparámetros del sampler (temperatures, walkers, steps), y una posición inicial.

In [10]:
def log_like(x, ivar):
    # likelihood function
    return -0.5 * np.sum(ivar * x ** 2)

def log_prior(x):
    # uninformative prior
    return 0.0

ndim, nwalkers = 5, 100
ntemps = 5
ivar = 1. / np.random.rand(ndim)

# initial position
p0 = list(np.random.randn(10, nwalkers, ndim))

In [11]:
# iniciamos el sampler
sampler = reddemcee.PTSampler(nwalkers, ndim, log_like, log_prior,
                              ntemps=ntemps, adaptative=True, logl_args=[ivar])

In [12]:
s0 = sampler.run_mcmc(p0, 200, 2)

100%|██████████| 400/400 [00:04<00:00, 82.25it/s]


Debería ver que la barra llegó a 100% sin errores.
Esto basta para confirmar que no hubo problemas con la instalación de la librería.

### 2.3 EMPER0R

En este bloque haremos algo similar para EMPER0R. Haremos un run extremadamente corto, para verificar que todo funciona correctamente.

La barra de carga no funciona correctamente en el collab, para obtener un aproximado del tiempo que tardará en correr, use este test como benchmark.

Para un setup de [2, 50, 100, 2], como es en este caso, el número de iteraciones que probará es la multiplicación de estos números:
$2 \cdot 50 \cdot 100 \cdot 2 = 20,000$ valores distintos. Escalelo linealmente como muestra la siguiente tabla:



| Iteraciones (N) | Tiempo (s) |
|-----------------|------------|
| 20,000          | 40         |
| 40,000          | 80         |
| 80,000          | 160        |

Para que la librería funcione correctamente, es necesario agregar el path de ella para que python la reconozca correctamente (celda siguiente). Si aún así obtiene un error al ejecutar

```
sim.run_auto(setup, k_start=1, k_end=1)
```
Pruebe volviendo a correr la celda
```
sys.path.append('/content/drive/MyDrive/udp_astro_101')
```


In [17]:
import sys
sys.path.append('/content/drive/MyDrive/udp_astro_101')

In [18]:
sim = emp.Simulation()
sim.set_engine('reddemcee')
setup = np.array([2, 50, 100, 2])  # temperaturas, walkers, sweeps, steps

sim.load_data('51Peg')  # Target folder name in /datafiles/
sim.plot_all['paper_mode'] = True

                                                                                
                   ~~ Simulation Successfully Initialized ~~                    
                                                                                


                         Reading data from 51peg.vels
                          




In [19]:
sim.run_auto(setup, k_start=1, k_end=1)

                                       Offset block added, OffsetBlock          


                                       Jitter block added, JitterBlock          


                                   Keplerian block added, KeplerianBlock 1      





                                ~~ Setup Info ~~                                

Current Engine is            reddemcee 0.6.6

Number of cores is           2

Save location is             datalogs/51Peg/run_2/k1

Dynamical Criteria is        None

Posterior fit method is      Gaussian Mixtures

Limits constrain method is   sigma

Model Selection method is    BIC


                           ~~ Automatically Saving ~~                           


Logger       : ✔

Samples      : ✘

Posteriors   : ✔

Likelihoods  : ✔


Plots: Posteriors           : ✔

Plots: Keplerian Model      : ✔

Plots: Gaussian Mixture     : ✔

Plots: Parameter Histograms : ✔



                               ~~ Pre-Run Info ~~                               



Parame


100%|██████████| 9/9 [00:00<00:00, 12.45it/s]





                                 ~~ Best Fit ~~                                 



Parameter           Posterior                 Value (max)    Value (mean)    Sigma  Limits
------------------  ----------------------  -------------  --------------  -------  ---------------------------
Period 1            ~𝓝 (1055.436, 446.331)       1129.59         1055.44   446.331  [1.500000e+00 2.187042e+03]
Amplitude 1         ~𝓝 (12.122, 7.328)             18.699          12.122    7.328  [ 0.    79.833]
Phase 1             ~𝓝 (3.953, 0.927)               4.721           3.953    0.927  [0.    6.283]
Eccentricity 1      ~𝓝 (0.144, 0.087)               0.038           0.144    0.086  [0. 1.]
Longitude 1         ~𝓝 (1.477, 1.204)               0.452           1.477    1.204  [0.    6.283]

------------------  ------------------  -------------  -------------  -------  -------------
Semi-Major Axis 1   ~𝓝 (1.977, 0.623)           2.123          1.977    0.623  [   0. 1000.]
Minimum Mass 1      ~𝓝 


100%|██████████| 4/4 [00:28<00:00,  7.23s/it]

100%|██████████| 2/2 [00:04<00:00,  2.17s/it]




                            Plotting Histograms Plot                            


...



                           Plotting Gaussian Mixtures                           



100%|██████████| 7/7 [00:06<00:00,  1.10it/s]



Time RUN         :  00:00:41

Time POSTPROCESS :  00:00:00

Time CALCULATE GM:  00:00:00

Time Plot model      :  00:00:05

Time Plot posteriors :  00:00:34

Time Plot histograms :  00:00:06

Time Plot betas      :  00:00:00

Time Plot arviz      :  00:00:23

Time Plot GM         :  00:00:06
                                  Cleaning Run                                  



100%|██████████| 3/3 [00:00<00:00, 183.95it/s]



BIC condition met!!

present BIC < past BIC - 5
2648.448 < inf - 5


                                                                                
                            ~~ Run came to an end ~~                            
                                                                                
                                       
                                        


In [ ]:
Podrá encontrar todos los resultados en udp_astro_101/datalogs/51Peg/run_X, donde el X mayor corresponde al último run que haya hecho.
Las subcarpetas kX/ representan el número de keplerianas en el modelo.

## Preguntas a abordar

¿Qué forma tienen las estimaciones posteriores marginadas bajo diferentes supuestos de cómo se aplican los modelos de ruido?

¿Cómo funciona el muestreador al analizar un único conjunto de datos de series temporales PRV, frente a varios conjuntos de datos o todos los conjuntos de datos combinados?

¿Qué modelo de ruido funciona mejor al analizar estos datos? ¿Y por qué?

¿Qué antecedentes son los más adecuados para los parámetros necesarios para modelar dichos datos y descubrir planetas?

¿Cómo actúan las métricas aplicadas al incluir o no un modelo de planeta kepleriano en lugar de un modelo de ruido plano? ¿Cómo continúa esto cuando se avanza hacia múltiples planetas?

¿Qué métricas cree que son las más adecuadas para este tipo de análisis? Sugerencia: compare y analice las probabilidades extraídas de diferentes estimadores como BIC, AIC, RMS, etc., y explique lo que cree que está sucediendo.

¿Cómo se comparan las estimaciones de los parámetros finales con los valores determinados cuando se utiliza un procedimiento de ajuste más estándar de Lomb-Scargle y Kepleriano? Tenga en cuenta que puede utilizar herramientas como ExoStriker para esta parte, pero no es estrictamente necesario para hacer este comparación.

ExoStriker GitHub: https://github.com/3fon3fonov/exostriker

**Nota que no es necesario a responder de todos de las preguntas, son para dar ideas principalmente!!